In [7]:
#import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')

ModuleNotFoundError: No module named 'pandas'

In [10]:
#load data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv') 


NameError: name 'pd' is not defined

In [9]:
# Display data information
print("Training Data:")
print("Shape:", train_data.shape)
print("\nHead:")
print(train_data.head())
print("\nClass Distribution:")
print(train_data['target'].value_counts())

print("\nTest Data:")
print("Shape:", test_data.shape)
print("\nHead:")
print(test_data.head())

Training Data:


NameError: name 'train_data' is not defined